In [1]:
# importing relevant packages 

import torch 
from torchvision import models
import pandas as pd
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
import glob
import os
import cv2
import matplotlib.pyplot as plt 
import torchvision.transforms as transforms
from torchsummary import summary
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import json
from PIL import Image
import random
import albumentations as A
import glob
from shapely.geometry import Polygon

/home/dell/Desktop/xxxx/Uni/Kirigami_project/Keypoint_detection_notebooks/.Keypoint_detection_notebooks/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [ ]:


# x = [0, 1, 2, 3, 4, 5, 6, 5, 4, 3, 2, 1]
# y = [0, 1, 2, 2, 2, 1, 0, -1, -2, -2, -2, -1]

# points = list(zip(x,y))
# # points = [(0, 0), (1, 1), (2, 0), (1, -1)]
# polygon = Polygon(points)
# bounds = polygon.bounds
# width = bounds[2] - bounds[0]
# height = bounds[3] - bounds[1]
# aspect_ratio = width / height

# print(aspect_ratio)
# print(polygon.area)
# print(polygon.length)
# print(polygon.bounds)

In [2]:
class VGG_model(nn.Module):
    def __init__(self):
        super(VGG_model, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        )
        
        self.avgpool = nn.Sequential(
            nn.Conv2d(512,512, kernel_size=3, padding='same'),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512,50, kernel_size=3, padding='same'),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.AdaptiveAvgPool2d(output_size=(8,8))
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(3200, 300),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Dropout(0.3),
            nn.Linear(300, 32),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
        

In [3]:
model = VGG_model()
model.load_state_dict(torch.load('./Model_VGG_4_5_2000_13080_epoch'))
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

In [4]:
def predict_keypoints(img):
    img = cv2.resize(img, (224,224))
    img_tensor = transforms.ToTensor()(img)
    img_mean = img_tensor.mean(dim = (1,2))
    img_std = img_tensor.std(dim = (1,2))
        
    img_normalised = transforms.Normalize(img_mean, img_std)(img_tensor)
    img_normalised = img_normalised.to(device)
    
    key_points = model(img_normalised[None]).flatten().detach().cpu().numpy()
    
    return key_points

In [5]:
def plot_keypoints(img, keypoints):                                                             

    plt.imshow(img)

    x_points = keypoints[0::2]
    y_points = keypoints[1::2]
    plt.scatter(x_points*img.shape[1], y_points*img.shape[0], s = 4, c=(1,0,0))
    plt.show()

In [6]:
dataset_names = ['Grasp_dataset_orange', 'Grasp_dataset_pear', 'Grasp_dataset_banana', 'Grasp_dataset_plum', 'Grasp_dataset_egg', 'Grasp_dataset_strawberry', 'Grasp_dataset_chicken', 'Grasp_dataset_bayberry', 'Grasp_dataset_redgrape', 'Grasp_dataset_pistachio']
# dataset_names = ['Grasp_dataset_orange', 'Grasp_dataset_pear', 'Grasp_dataset_banana', 'Grasp_dataset_plum', 'Grasp_dataset_egg', 'Grasp_dataset_strawberry', 'Grasp_dataset_chicken', 'Grasp_dataset_bayberry', 'Grasp_dataset_redgrape']

header = ['pressure_reading_1', 'pressure_reading_2', 'force_reading_1', 'force_reading_2', 'kirigami_area', 'kirigami_perimeter', 'kirigami_width', 'kirigami_height', 'kirigami_aspect_ratio', 'hole_area', 'hole_perimeter', 'hole_width', 'hole_height', 'hole_aspect_ratio', 'p1_x', 'p1_y', 'p2_x', 'p2_y', 'p3_x', 'p3_y', 'p4_x', 'p4_y', 'p5_x', 'p5_y', 'p6_x', 'p6_y', 'p7_x', 'p7_y', 'p8_x', 'p8_y', 'p9_x', 'p9_y', 'p10_x', 'p10_y', 'p11_x', 'p11_y', 'p12_x', 'p12_y', 'p13_x', 'p13_y', 'p14_x', 'p14_y', 'p15_x', 'p15_y', 'p16_x', 'p16_y', 'label']

current_dir = os. getcwd()

pressure_reading_1_arr = []
pressure_reading_2_arr = []
force_reading_1_arr = []
force_reading_2_arr = []

kirigami_area_arr = []
kirigami_perimeter_arr = []
kirigami_width_arr = []
kirigami_height_arr = []
kirigami_aspect_ratio_arr = []
hole_area_arr = []
hole_perimeter_arr = []
hole_width_arr = []
hole_height_arr = []
hole_aspect_ratio_arr = []

p1_x_arr = []
p1_y_arr = []
p2_x_arr = []
p2_y_arr = []
p3_x_arr = []
p3_y_arr = []
p4_x_arr = []
p4_y_arr = []
p5_x_arr = []
p5_y_arr = []
p6_x_arr = []
p6_y_arr = []
p7_x_arr = []
p7_y_arr = []
p8_x_arr = []
p8_y_arr = []
p9_x_arr = []
p9_y_arr = []
p10_x_arr = []
p10_y_arr = []
p11_x_arr = []
p11_y_arr = []
p12_x_arr = []
p12_y_arr = []
p13_x_arr = []
p13_y_arr = []
p14_x_arr = []
p14_y_arr = []
p15_x_arr = []
p15_y_arr = []
p16_x_arr = []
p16_y_arr = []
label_arr = []


for f in dataset_names:
    
    image_dir = os.path.join(current_dir + '/Grasp_dataset_2', f)
    data_dir = os.path.join(image_dir, f + '.csv')
    df = pd.read_csv(data_dir)
    pressure_reading_1_arr = np.concatenate((pressure_reading_1_arr, df['pressure_reading_1'].values))
    pressure_reading_2_arr = np.concatenate((pressure_reading_2_arr, df['pressure_reading_2'].values))
    force_reading_1_arr = np.concatenate((force_reading_1_arr, df['force_reading_1'].values))
    force_reading_2_arr = np.concatenate((force_reading_2_arr, df['force_reading_2'].values))
    label_arr = np.concatenate((label_arr, df['label'].values))
    
    j = 0
    
    while j < df.shape[0]:
        image = cv2.imread(os.path.join(image_dir, df.iloc[j]['image_name']))
        keypoints = predict_keypoints(image)
        
#         plot_keypoints(image, keypoints)
        
#         if j == 20:
#             plot_keypoints(image, keypoints)
        
        kirigami_x_points = keypoints[0:12:2]
        kirigami_y_points = keypoints[1:12:2]
        
#         plt.imshow(image)

#         plt.scatter(kirigami_x_points*image.shape[1], kirigami_y_points*image.shape[0], s = 4, c=(1,0,0))
#         plt.show()
        
        kirigami_points = list(zip(kirigami_x_points, kirigami_y_points))
        
        kirigami_polygon = Polygon(kirigami_points)
        kirigami_area = kirigami_polygon.area
        kirigami_perimeter = kirigami_polygon.length
        kirigami_bounds = kirigami_polygon.bounds
        kirigami_width = kirigami_bounds[2] - kirigami_bounds[0]
        kirigami_height = kirigami_bounds[3] - kirigami_bounds[1]
        kirigami_aspect_ratio = kirigami_width / kirigami_height
        
        kirigami_area_arr.append(kirigami_area)
        kirigami_perimeter_arr.append(kirigami_perimeter)
        kirigami_width_arr.append(kirigami_width)
        kirigami_height_arr.append(kirigami_height)
        kirigami_aspect_ratio_arr.append(kirigami_aspect_ratio)
        
        
        hole_x_points = np.array([keypoints[2], keypoints[8]] + keypoints[12::2].tolist())
        hole_y_points = np.array([keypoints[3], keypoints[9]] + keypoints[13::2].tolist())
        
#         plt.imshow(image)

#         plt.scatter(hole_x_points*image.shape[1], hole_y_points*image.shape[0], s = 4, c=(1,0,0))
#         plt.show()
        
        hole_points = list(zip(hole_x_points, hole_y_points))
        
        hole_polygon = Polygon(hole_points)
        hole_area = hole_polygon.area
        hole_perimeter = hole_polygon.length
        hole_bounds = hole_polygon.bounds
        hole_width = hole_bounds[2] - hole_bounds[0]
        hole_height = hole_bounds[3] - hole_bounds[1]
        hole_aspect_ratio = hole_width / hole_height
        
        hole_area_arr.append(hole_area)
        hole_perimeter_arr.append(hole_perimeter)
        hole_width_arr.append(hole_width)
        hole_height_arr.append(hole_height)
        hole_aspect_ratio_arr.append(hole_aspect_ratio)
        
        p1_x_arr.append(keypoints[0])
        p1_y_arr.append(keypoints[1])
        p2_x_arr.append(keypoints[2])
        p2_y_arr.append(keypoints[3])
        p3_x_arr.append(keypoints[4])
        p3_y_arr.append(keypoints[5])
        p4_x_arr.append(keypoints[6])
        p4_y_arr.append(keypoints[7])
        p5_x_arr.append(keypoints[8])
        p5_y_arr.append(keypoints[9])
        p6_x_arr.append(keypoints[10])
        p6_y_arr.append(keypoints[11])
        p7_x_arr.append(keypoints[12])
        p7_y_arr.append(keypoints[13])
        p8_x_arr.append(keypoints[14])
        p8_y_arr.append(keypoints[15])
        p9_x_arr.append(keypoints[16])
        p9_y_arr.append(keypoints[17])
        p10_x_arr.append(keypoints[18])
        p10_y_arr.append(keypoints[19])
        p11_x_arr.append(keypoints[20])
        p11_y_arr.append(keypoints[21])
        p12_x_arr.append(keypoints[22])
        p12_y_arr.append(keypoints[23])
        p13_x_arr.append(keypoints[24])
        p13_y_arr.append(keypoints[25])
        p14_x_arr.append(keypoints[26])
        p14_y_arr.append(keypoints[27])
        p15_x_arr.append(keypoints[28])
        p15_y_arr.append(keypoints[29])
        p16_x_arr.append(keypoints[30])
        p16_y_arr.append(keypoints[31])
        
        j = j + 1
        
#         if j == 2:
#             break
#     break

grasp_dataset = pd.concat([pd.Series(pressure_reading_1_arr), pd.Series(pressure_reading_2_arr), pd.Series(force_reading_1_arr), pd.Series(force_reading_2_arr), pd.Series(kirigami_area_arr), pd.Series(kirigami_perimeter_arr), pd.Series(kirigami_width_arr), pd.Series(kirigami_height_arr), pd.Series(kirigami_aspect_ratio_arr), pd.Series(hole_area_arr), pd.Series(hole_perimeter_arr), pd.Series(hole_width_arr), pd.Series(hole_height_arr), pd.Series(hole_aspect_ratio_arr), pd.Series(p1_x_arr), pd.Series(p1_y_arr), pd.Series(p2_x_arr), pd.Series(p2_y_arr), pd.Series(p3_x_arr), pd.Series(p3_y_arr), pd.Series(p4_x_arr), pd.Series(p4_y_arr), pd.Series(p5_x_arr), pd.Series(p5_y_arr), pd.Series(p6_x_arr), pd.Series(p6_y_arr), pd.Series(p7_x_arr), pd.Series(p7_y_arr), pd.Series(p8_x_arr), pd.Series(p8_y_arr), pd.Series(p9_x_arr), pd.Series(p9_y_arr), pd.Series(p10_x_arr), pd.Series(p10_y_arr), pd.Series(p11_x_arr), pd.Series(p11_y_arr), pd.Series(p12_x_arr), pd.Series(p12_y_arr), pd.Series(p13_x_arr), pd.Series(p13_y_arr), pd.Series(p14_x_arr), pd.Series(p14_y_arr), pd.Series(p15_x_arr), pd.Series(p15_y_arr), pd.Series(p16_x_arr), pd.Series(p16_y_arr), pd.Series(label_arr)], axis=1, keys=header)


In [7]:
pd.options.display.max_columns = None
print(grasp_dataset.head(2))

   pressure_reading_1  pressure_reading_2  force_reading_1  force_reading_2  \
0                92.5                91.7            414.0            408.5   
1                89.7                93.3            448.0            426.0   

   kirigami_area  kirigami_perimeter  kirigami_width  kirigami_height  \
0       0.477233            2.592495        0.704943         0.947765   
1       0.475765            2.583629        0.710457         0.938301   

   kirigami_aspect_ratio  hole_area  hole_perimeter  hole_width  hole_height  \
0               0.743795   0.060664        3.166254    0.161493     0.947765   
1               0.757174   0.062354        3.147166    0.163671     0.938301   

   hole_aspect_ratio      p1_x      p1_y      p2_x      p2_y      p3_x  \
0           0.170393  0.137714  0.277483  0.489576  0.013344  0.831671   
1           0.174433  0.133174  0.292527  0.490042  0.019566  0.830556   

       p3_y      p4_x      p4_y      p5_x      p5_y      p6_x      p6_y  \
0  

In [8]:
grasp_dataset

,pressure_reading_1,pressure_reading_2,force_reading_1,force_reading_2,kirigami_area,kirigami_perimeter,kirigami_width,kirigami_height,kirigami_aspect_ratio,hole_area,hole_perimeter,hole_width,hole_height,hole_aspect_ratio,p1_x,p1_y,p2_x,p2_y,p3_x,p3_y,p4_x,p4_y,p5_x,p5_y,p6_x,p6_y,p7_x,p7_y,p8_x,p8_y,p9_x,p9_y,p10_x,p10_y,p11_x,p11_y,p12_x,p12_y,p13_x,p13_y,p14_x,p14_y,p15_x,p15_y,p16_x,p16_y,label
0,92.5,91.7,414.0,408.5,0.477233,2.592495,0.704943,0.947765,0.743795,0.060664,3.166254,0.161493,0.947765,0.170393,0.137714,0.277483,0.489576,0.013344,0.831671,0.296866,0.842657,0.673206,0.495992,0.961110,0.148972,0.754630,0.443099,0.216665,0.415440,0.383071,0.404647,0.519336,0.418402,0.649931,0.450821,0.798194,0.532054,0.807107,0.555927,0.650187,0.566140,0.522067,0.552346,0.388728,0.528648,0.226195,orange
1,89.7,93.3,448.0,426.0,0.475765,2.583629,0.710457,0.938301,0.757174,0.062354,3.147166,0.163671,0.938301,0.174433,0.133174,0.292527,0.490042,0.019566,0.830556,0.278508,0.843630,0.680346,0.491899,0.957867,0.138661,0.726935,0.444310,0.216582,0.415869,0.382468,0.404388,0.518507,0.417284,0.648944,0.447945,0.797169,0.531323,0.805627,0.556485,0.648771,0.568058,0.521511,0.555142,0.388794,0.530346,0.227650,orange
2,86.4,95.7,451.8,426.9,0.489883,2.619521,0.719269,0.946502,0.759923,0.063615,3.169986,0.167929,0.946502,0.177421,0.124487,0.282194,0.490909,0.017936,0.832869,0.275265,0.843756,0.687739,0.495769,0.964438,0.131904,0.734036,0.442059,0.218476,0.412865,0.386329,0.402257,0.523790,0.416887,0.653943,0.450033,0.800964,0.535091,0.809223,0.559454,0.653806,0.570186,0.526559,0.557050,0.392467,0.532980,0.229078,orange
3,79.0,102.2,463.9,425.3,0.480929,2.602163,0.712847,0.952921,0.748065,0.062412,3.177873,0.160257,0.952921,0.168175,0.126096,0.326686,0.488317,0.017855,0.825797,0.267587,0.838943,0.704834,0.492908,0.970775,0.129704,0.714765,0.447426,0.218678,0.421780,0.384694,0.411901,0.520693,0.425393,0.651769,0.453223,0.801325,0.534909,0.809144,0.560583,0.651304,0.572158,0.523461,0.558460,0.390657,0.531070,0.229785,orange
4,69.0,104.2,465.3,414.7,0.468220,2.566274,0.702698,0.930650,0.755062,0.060498,3.121641,0.154418,0.930650,0.165925,0.128294,0.343078,0.488589,0.032270,0.813509,0.232563,0.830992,0.711803,0.494560,0.962920,0.132716,0.712219,0.450608,0.220885,0.426818,0.385017,0.417513,0.522805,0.429782,0.653641,0.456140,0.800150,0.535668,0.805884,0.560082,0.650654,0.571931,0.523777,0.559604,0.390745,0.531838,0.232188,orange
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,24.5,18.8,146.0,149.3,0.386810,2.317078,0.704574,0.752952,0.935750,0.038153,2.705498,0.102786,0.752952,0.136511,0.133096,0.349935,0.474901,0.133627,0.833403,0.324972,0.837670,0.686706,0.496723,0.886579,0.148371,0.706866,0.446115,0.232907,0.438198,0.382879,0.434341,0.517846,0.445863,0.650487,0.464919,0.790408,0.524873,0.802771,0.533135,0.657993,0.537127,0.525593,0.525973,0.391765,0.508132,0.245157,pistachio
496,7.0,8.2,131.3,130.0,0.392691,2.338247,0.721115,0.740438,0.973903,0.039952,2.672491,0.106756,0.740438,0.144179,0.117137,0.357639,0.472127,0.137911,0.831694,0.317375,0.838252,0.682077,0.491057,0.878349,0.125108,0.710053,0.440946,0.234037,0.432894,0.381118,0.429368,0.517106,0.439632,0.649965,0.457109,0.785624,0.521926,0.796515,0.531028,0.655080,0.536123,0.523414,0.525791,0.389731,0.508717,0.245428,pistachio
497,7.8,5.8,128.9,129.6,0.373358,2.277838,0.690728,0.731037,0.944860,0.036855,2.643723,0.098917,0.731037,0.135311,0.141177,0.358059,0.473257,0.148265,0.825731,0.321198,0.831904,0.679733,0.491297,0.879302,0.145557,0.713789,0.444249,0.238631,0.437572,0.384981,0.433996,0.520413,0.443610,0.652832,0.459168,0.788260,0.520714,0.797776,0.528367,0.656521,0.532913,0.525090,0.522662,0.391596,0.507469,0.247444,pistachio
498,6.0,7.6,129.5,128.4,0.377299,2.293609,0.712830,0.716831,0.994419,0.038350,2.621334,0.101684,0.716831,0.141852,0.1293

In [145]:
grasp_data = grasp_dataset.drop(columns=['label'], axis=1)
# grasp_data = grasp_dataset.drop(columns=['label', 'p1_x', 'p1_y', 'p2_x', 'p2_y', 'p3_x', 'p3_y', 'p4_x', 'p4_y', 'p5_x', 'p5_y', 'p6_x', 'p6_y', 'p7_x', 'p7_y', 'p8_x', 'p8_y', 'p9_x', 'p9_y', 'p10_x', 'p10_y', 'p11_x', 'p11_y', 'p12_x', 'p12_y', 'p13_x', 'p13_y', 'p14_x', 'p14_y', 'p15_x', 'p15_y', 'p16_x', 'p16_y'], axis=1)
grasp_label = grasp_dataset['label']

print(grasp_data.head(2))
print(grasp_label.head(2))

   pressure_reading_1  pressure_reading_2  force_reading_1  force_reading_2  \
0                92.5                91.7            414.0            408.5   
1                89.7                93.3            448.0            426.0   

   kirigami_area  kirigami_perimeter  kirigami_width  kirigami_height  \
0       0.477233            2.592495        0.704943         0.947765   
1       0.475765            2.583629        0.710457         0.938301   

   kirigami_aspect_ratio  hole_area  hole_perimeter  hole_width  hole_height  \
0               0.743795   0.060664        3.166254    0.161493     0.947765   
1               0.757174   0.062354        3.147166    0.163671     0.938301   

   hole_aspect_ratio      p1_x      p1_y      p2_x      p2_y      p3_x  \
0           0.170393  0.137714  0.277483  0.489576  0.013344  0.831671   
1           0.174433  0.133174  0.292527  0.490042  0.019566  0.830556   

       p3_y      p4_x      p4_y      p5_x      p5_y      p6_x      p6_y  \
0  

In [148]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier

data_train, data_test, label_train, label_test = train_test_split(grasp_data, grasp_label, test_size=0.2, random_state=np.random.randint(100))

# print(X_train.head(3))

# clf = KNeighborsClassifier(n_neighbors=11)

# knn.fit(X_train, y_train)

# y_pred = knn.predict(X_test)

# accuracy = knn.score(X_test, y_test)
# print('Accuracy:', accuracy)
clf = RandomForestClassifier(n_estimators=100, max_depth=100, random_state=np.random.randint(100))
# clf = HistGradientBoostingClassifier(max_iter=200, random_state=np.random.randint(100))
# clf = GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, max_depth=1, random_state=0)
# clf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)],voting='hard')

# clf = RandomForestClassifier(n_estimators=500, max_depth=50, random_state=np.random.randint(100))

num_folds = 10
cv_method = KFold(n_splits=num_folds, shuffle=True, random_state=np.random.randint(100))
cv_results = cross_val_score(clf, grasp_data, grasp_label, cv=cv_method, scoring='accuracy')

print('Cross-validation results:', cv_results)
print('Average accuracy:', cv_results.mean())

# clf.fit(data_train, label_train)
# label_pred = clf.predict(data_test)
# cm = confusion_matrix(label_test, label_pred)

# short_dataset_names = ['orange', 'pear', 'banana', 'plum', 'egg', 'straw', 'chicken', 'bayberry', 'grape', 'pistachio']
# # short_dataset_names = ['orange', 'pear', 'banana', 'plum', 'egg', 'straw', 'chicken', 'bayberry', 'grape']


# cm_df = pd.DataFrame(cm, index=short_dataset_names, columns=short_dataset_names)

# print(cm_df)
# print(cm)

# accuracy = clf.score(data_test, label_test)
# print("Accuracy:", accuracy)

Cross-validation results: [0.72 0.82 0.84 0.82 0.74 0.78 0.78 0.84 0.9  0.7 ]
Average accuracy: 0.794
